In [2]:
import numpy as np

def GSO(B): #グラム・シュミット直交化
  n = len(B)
  B_star = np.zeros_like(B, dtype=float)
  mu = np.eye(n)

  for i in range(n):
      B_star[i] = B[i]
      for j in range(i):
        mu[i, j] = np.dot(B[i], B_star[j]) / np.dot(B_star[j], B_star[j])
        B_star[i] -= mu[i, j] * B_star[j]

  return B_star, mu

#(i,j)成分に関する部分サイズ簡約
def Partial_Size_Reduce(B, mu, i, j):
  if abs(mu[i,j]) > 0.5:
    q = round(mu[i, j])
    #|mu[i,j]| <= 0.5 となるようにB[i]を取り直す
    B[i] -= q * B[j]
    #GSO係数の更新
    for l in range(j+1):
      mu[i,l] -= q * mu[j,l]

#Lovasz条件(True or Falseを出力)
def lovasz_condition(B_star_norm, mu, delta, i):
  return B_star_norm[i] >= (delta - mu[i, i-1]**2) *B_star_norm[i-1]

#効率的なGSO情報更新
def GSOUpdate_LLL(B_star_norm, mu, k) :
  #更新されると困る繰り返し用いる値を固定
  nu = mu[k,k-1]
  B_temp = B_star_norm[k] + nu**2 * B_star_norm[k-1]
  #(i,j)=(k,k-1)に関する値を更新
  mu[k,k-1] = nu * B_star_norm[k-1] / B_temp
  B_star_norm[k] = B_star_norm[k] * B_star_norm[k-1] / B_temp
  B_star_norm[k-1] = B_temp
  #(i,j)=(k,j),(k-1,j)に関する値を更新
  for j in range(0,k-1):
    mu[k-1,j], mu[k,j] = mu[k,j], mu[k-1,j]
  #(i,j)=(i,k), (i,k-1)に関する値を更新
  for i in range(k+1, len(B_star_norm)):
    t = mu[i,k]
    mu[i,k] = mu[i,k-1] - nu * t
    mu[i,k-1] = t + mu[k,k-1] * mu[i,k]

  return B_star_norm, mu

#LLL基底簡約アルゴリズム
def LLL(B, delta=0.75):
  n = len(B)
  #Bの型をnp.arrayに
  B = np.array(B, dtype=int)
  #BのGSO情報を取得
  B_star, mu = GSO(B)
  #B_starの2乗ノルムを格納する配列を作成（初期値として0）
  B_star_norm = np.zeros(n)
  #B_star_binormにBの情報を入れる
  for i in range(n):
    B_star_norm[i] = np.linalg.norm(B_star[i])**2

  #行の初期値（mu[0,0]=1なので次の行からスタート）
  k = 1
  while k < n:
    #(k,j)成分に関して部分サイズ簡約
    for j in range(k-1, -1, -1):
      Partial_Size_Reduce(B, mu, k, j)

    #kにおけるLovasz条件の判定
    if lovasz_condition(B_star_norm, mu, delta, k):
      #もしOKならkを更新
      k += 1
    else:
      #もしFalseなら隣り合う基底ベクトルを交換
      B[[k-1,k]] = B[[k,k-1]]
      #↑素朴な方法（temp使う or a , b = b , a みたいな感じ）だと上手くいかなかったのでchatGPTに聞いた
      #GSO情報を更新
      B_star_norm, mu = GSOUpdate_LLL(B_star_norm, mu, k)
      #kの値を更新
      k = max(k-1, 1)

  return B

#LLL基底簡約アルゴリズムの挙動確認版
def test_LLL(B, delta=0.75):
  n = len(B)
  #Bの型をndarrayに
  B = np.array(B, dtype=int)
  #BのGSO情報を取得
  B_star, mu = GSO(B)
  #B_starの2乗ノルムを格納する配列を作成（初期値として0）
  B_star_norm = np.zeros(n)
  #B_star_binormにBの情報を入れる
  for i in range(n):
    B_star_norm[i] = np.linalg.norm(B_star[i])**2
  #GSOベクトル行列, GSO係数行列, GSOベクトルの二乗ノルムを表示
  print('---')
  print('Initial condition :')
  print('GSO_vectors')
  print(B_star)
  print('GSO_coefficients')
  print(mu)
  print('GSO_vector_norm')
  print(B_star_norm)

  #行の初期値（mu[0,0]=1なので次の行からスタート）
  k = 1

  while k < n:
    print('---')
    print('k =',k)
    #(k,j)成分に関して部分サイズ簡約
    for j in range(k-1, -1, -1):
      Partial_Size_Reduce(B, mu, k, j)

      #基底簡約後の基底行列, GSO係数行列, GSOベクトルのノルムを表示
      print('---')
      print('Partial_Size_Reduction')
      print('(k,j) : ')
      print(k,j)
      print('Basis matrix : B')
      print(B)
      print('GSO coefficients : mu')
      print(mu)
      print('GSO vector norm : B_star_norm')
      print(B_star_norm)

    #Lovasz条件の表示
    print('---')
    print('Lovasz condition at k =',k,':',lovasz_condition(B_star_norm, mu, delta, k))

    #kにおけるLovasz条件の判定
    if lovasz_condition(B_star_norm, mu, delta, k):
      #もしOKならkを更新
      k += 1
      print('k += 1')

    else:
      #もしFalseなら隣り合う基底ベクトルを交換
      print('---')
      print(B)
      B[[k-1,k]] = B[[k,k-1]]
      #↑素朴な方法（temp使う or a , b = b , a みたいな感じ）だと上手くいかなかったのでchatGPTに聞いた
      print('---')
      print('swap(B[k-1], B[k])')
      print('---')
      print(B)
      #GSO情報を更新
      B_star_norm, mu = GSOUpdate_LLL(B_star_norm, mu, k)
      print('---')
      print('update GSO info')
      print('GSO coefficients : mu')
      print(mu)
      print('GSO vector norm : B_star_norm')
      print(B_star_norm)

      #kの値を更新
      k = max(k-1, 1)
      print('---')
      print('update k = max(k-1,1) =',k)

  return B

In [3]:
# 標準入力から行列を生成
def input_basis_form():
  print("Enter the dimension of the lattice.")
  print("Enter each basis vector of the lattice")
  n = int(input())
  B = np.array([list(map(int,input().split())) for _ in range(n)])
  return B

# 標準入力でLLLに入れる行列を入力する
def test_LLL_with_stdin():
  B = input_basis_form()
  print("Enter the value of delta")
  delta = float(input())
  test_LLL(B, delta)

# ランダムな整数係数フルランク行列を生成するアルゴリズム（やっつけバージョン）
def create_randint_fullrank_matrix():
  print("Enter the size of matrix")
  s = list(map(int, input().split()))
  row = s[0]
  col = s[1]
  while True:
    rand = np.random.randint(-1000, 1000, size=(row, col))
    if np.linalg.matrix_rank(rand) == row:
      return rand
    else:
      continue

# ランダムな整数係数フルランク行列にLLLを適用する
def test_LLL_with_random_matrix():
  B = create_randint_fullrank_matrix()
  print("The basis matrix is")
  print(B)
  print("---")
  print("Enter the value of delta")
  delta = float(input())
  print("---")
  print("Do you want to check each step?[y/n]")
  answer = input()
  if answer == "y":
    print(test_LLL(B, delta))
  else:
    print("the LLL reduced basis is")
    print(LLL(B, delta))

In [6]:
B1 = np.array([
     [9, 2, 7],
    [8, 6, 1],
    [3, 2, 6],
])
print("Original Basis:")
print(B1)

B1_LLL = LLL(B1, delta=0.99)
print("LLL-reduced Basis:")
print(B1_LLL)

Original Basis:
[[9 2 7]
 [8 6 1]
 [3 2 6]]
LLL-reduced Basis:
[[ 6  0  1]
 [ 3 -2 -5]
 [ 2  6  0]]


In [7]:
#You can also check the behavior at each step of LLL
print(test_LLL(B1, delta=0.99))

---
Initial condition :
GSO_vectors
[[ 9.          2.          7.        ]
 [ 1.8880597   4.64179104 -3.75373134]
 [-1.53816867  1.80734818  1.46126023]]
GSO_coefficients
[[ 1.          0.          0.        ]
 [ 0.67910448  1.          0.        ]
 [ 0.54477612 -0.19322292  1.        ]]
GSO_vector_norm
[134.          39.20149254   7.76775176]
---
k = 1
---
Partial_Size_Reduction
(k,j) : 
1 0
Basis matrix : B
[[ 9  2  7]
 [-1  4 -6]
 [ 3  2  6]]
GSO coefficients : mu
[[ 1.          0.          0.        ]
 [-0.32089552  1.          0.        ]
 [ 0.54477612 -0.19322292  1.        ]]
GSO vector norm : B_star_norm
[134.          39.20149254   7.76775176]
---
Lovasz condition at k = 1 : False
---
[[ 9  2  7]
 [-1  4 -6]
 [ 3  2  6]]
---
swap(B[k-1], B[k])
---
[[-1  4 -6]
 [ 9  2  7]
 [ 3  2  6]]
---
update GSO info
GSO coefficients : mu
[[ 1.          0.          0.        ]
 [-0.81132075  1.          0.        ]
 [-0.58490566  0.48277175  1.        ]]
GSO vector norm : B_star_norm
[53.  

In [7]:
B2 = np.array([
  [-2, 7, 7, -5],
  [3, -2, 6, -1],
  [2, -8, -9, -7],
  [8, -9, 6, -4],
])
print("Original Basis:")
print(B2)

B2_LLL = LLL(B2, delta=0.75)
print("LLL-reduced Basis:")
print(B2_LLL)

Original Basis:
[[-2  7  7 -5]
 [ 3 -2  6 -1]
 [ 2 -8 -9 -7]
 [ 8 -9  6 -4]]
LLL-reduced Basis:
[[ 2  3  1  1]
 [-2  2  3 -3]
 [ 2  0 -2 -4]
 [ 3 -2  6 -1]]


In [8]:
print(test_LLL(B2, delta=0.75))

---
Initial condition :
GSO_vectors
[[-2.00000000e+00  7.00000000e+00  7.00000000e+00 -5.00000000e+00]
 [ 3.42519685e+00 -3.48818898e+00  4.51181102e+00  6.29921260e-02]
 [ 1.10104964e+00 -3.64543676e+00 -3.50827255e+00 -1.04556129e+01]
 [ 6.02114788e-01  3.55156871e-01 -1.82534113e-01  8.25946212e-04]]
GSO_coefficients
[[ 1.          0.          0.          0.        ]
 [ 0.21259843  1.          0.          0.        ]
 [-0.69291339 -0.14214553  1.          0.        ]
 [-0.13385827  1.93435332  0.4583152   1.        ]]
GSO_vector_norm
[127.          44.25984252 136.12933642   0.52199801]
---
k = 1
---
Partial_Size_Reduction
(k,j) : 
1 0
Basis matrix : B
[[-2  7  7 -5]
 [ 3 -2  6 -1]
 [ 2 -8 -9 -7]
 [ 8 -9  6 -4]]
GSO coefficients : mu
[[ 1.          0.          0.          0.        ]
 [ 0.21259843  1.          0.          0.        ]
 [-0.69291339 -0.14214553  1.          0.        ]
 [-0.13385827  1.93435332  0.4583152   1.        ]]
GSO vector norm : B_star_norm
[127.          44.

In [9]:
# 標準入力で色々試してみよう
test_LLL_with_stdin()

Enter the dimension of the lattice.
Enter each basis vector of the lattice
Enter the value of delta
---
Initial condition :
GSO_vectors
[[ 1.00000000e+00  5.00000000e+00  7.00000000e+00  2.30000000e+01]
 [ 2.33609272e+00  2.86804636e+01  6.35264901e+00 -8.26986755e+00]
 [-1.30224439e+01  2.84834901e+01 -9.39192893e+01  2.29582617e+01]
 [-2.52386663e-01  1.13481769e-02  3.77102994e-02 -2.97070945e-03]]
GSO_coefficients
[[ 1.          0.          0.          0.        ]
 [ 3.66390728  1.          0.          0.        ]
 [ 0.96688742  0.02378179  1.          0.        ]
 [ 5.0513245   1.07968385 -0.35928898  1.        ]]
GSO_vector_norm
[6.04000000e+02 9.36773179e+02 1.03288079e+04 6.52587007e-02]
---
k = 1
---
Partial_Size_Reduction
(k,j) : 
1 0
Basis matrix : B
[[  1   5   7  23]
 [  2  27   4 -16]
 [-12  34 -87  45]
 [ 12  46  76  99]]
GSO coefficients : mu
[[ 1.          0.          0.          0.        ]
 [-0.33609272  1.          0.          0.        ]
 [ 0.96688742  0.02378179  

ランダムなフルランク行列に対するLLL

In [5]:
test_LLL_with_random_matrix()

Enter the size of matrix
The basis matrix is
[[-477 -991  481 ...  754  672  191]
 [  33   51  507 ...  935 -584  526]
 [ 530  721  883 ...  622  738  186]
 ...
 [ 636  506  422 ...   67 -433  840]
 [ 319  923  512 ...  783  429  447]
 [  65 -286 -831 ... -849 -187 -866]]
---
Enter the value of delta
---
Do you want to check each step?[y/n]
the LLL reduced basis is
[[ -590   305  -362 ...   517   821  -166]
 [ -591  -424  -107 ...   970   278  -393]
 [ -577   889    50 ...  -395  -462  -648]
 ...
 [  461 -1013   -39 ...  2799  1714   632]
 [  993 -2770   423 ... -1298  -859   824]
 [  -80   396   610 ...   301   224  -476]]
